# Entrega laboratorio 7
## Integrantes 

- Javier Steven Barrera Toro - 202214779
- Julian Santiago Rolon Toloza - 202215839

In [1]:
from matplotlib import pyplot as plt
from typing import List, Tuple, NewType
import numpy as np
import pandas as pd
import sympy as sp
import random
import math
import copy
import tabulate
from matplotlib.colors import ListedColormap

np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Fase 1: Modelado de llegadas y salidas

## Extracción de tasas de llegada

## Modelado de comportamiento por tipo de usuario

# Fase 2: Implementación de la simulación en SimPy
## Definir las clases y los recursos
## Implementación de procesos
## Recopilación de estadísticas

# Fase 3: Validación y verificación
## Comparacón con datos reales
## Análisis de sensibilidad

# Fase 4: Experimentación de escenarios
## Diseño de experimentos
## Ejecución de escenarios

# Fase 5: Análisis y recomendaciones
## Análisis comparativo
## Elaboración de recomandaciones